<a href="https://colab.research.google.com/github/sanju053/MultiGPU/blob/master/TestWithCPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os.path
import pandas
import time
import xgboost as xgb
import sys
from dask.distributed import Client, wait
import dask_xgboost

import dask

import matplotlib.pyplot as plt
%matplotlib inline 



In [0]:
if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    from urllib import urlretrieve

data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz"
dmatrix_train_filename = "higgs_train.dmatrix"
dmatrix_test_filename = "higgs_test.dmatrix"
csv_filename = "HIGGS.csv.gz"
train_rows = 10500000
test_rows = 500000
num_round = 1000

plot = True

# return xgboost dmatrix
def load_higgs():
    if os.path.isfile(dmatrix_train_filename) and os.path.isfile(dmatrix_test_filename):           
        dtrain = xgb.DMatrix(dmatrix_train_filename)
        dtest = xgb.DMatrix(dmatrix_test_filename)
        if dtrain.num_row() == train_rows and dtest.num_row() == test_rows:
            print("Loading cached dmatrix...")
            return dtrain, dtest

    if not os.path.isfile(csv_filename):
        print("Downloading higgs file...")
        urlretrieve(data_url, csv_filename)

    df_higgs_train = pandas.read_csv(csv_filename, dtype=np.float32, 
                                     nrows=train_rows, header=None)
    dtrain = xgb.DMatrix(df_higgs_train.ix[:, 1:29], df_higgs_train[0])
    dtrain.save_binary(dmatrix_train_filename)
    df_higgs_test = pandas.read_csv(csv_filename, dtype=np.float32, 
                                    skiprows=train_rows, nrows=test_rows, 
                                    header=None)
    dtest = xgb.DMatrix(df_higgs_test.ix[:, 1:29], df_higgs_test[0])
    dtest.save_binary(dmatrix_test_filename)

    return dtrain, dtest

In [0]:
dtrain, dtest = load_higgs()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  dat

In [0]:
datasnap = pandas.read_csv(csv_filename, dtype=np.float32, 
                                     nrows=10, header=None)

In [0]:
print(datasnap)

    0         1         2         3   ...        25        26        27        28
0  1.0  0.869293 -0.635082  0.225690  ...  0.920005  0.721657  0.988751  0.876678
1  1.0  0.907542  0.329147  0.359412  ...  0.978098  0.779732  0.992356  0.798343
2  1.0  0.798835  1.470639 -1.635975  ...  0.951331  0.803252  0.865924  0.780118
3  0.0  1.344385 -0.876626  0.935913  ...  0.728281  0.869200  1.026736  0.957904
4  1.0  1.105009  0.321356  1.522401  ...  0.838085  1.133295  0.872245  0.808487
5  0.0  1.595839 -0.607811  0.007075  ...  0.789176  0.430553  0.961357  0.957818
6  1.0  0.409391 -1.884684 -1.027292  ...  0.545045  0.698653  0.977314  0.828786
7  1.0  0.933895  0.629130  0.527535  ...  0.783300  0.849195  0.894356  0.774879
8  1.0  1.405144  0.536603  0.689554  ...  1.045352  1.542972  3.534827  2.740754
9  1.0  1.176566  0.104161  1.397002  ...  1.103696  0.849140  0.937104  0.812364

[10 rows x 29 columns]


In [0]:
print("Training with CPU ...")
param = {}
param['objective'] = 'binary:logitraw'
param['eval_metric'] = 'error'
param['silent'] = 1
param['tree_method'] = 'hist'

tmp = time.time()
cpu_res = {}
xgb.train(param, dtrain, num_round, evals=[(dtest, "test")], 
          evals_result=cpu_res)
cpu_time = time.time() - tmp
print("CPU Training Time: %s seconds" % (str(cpu_time)))

Training with CPU ...
[0]	test-error:0.529014
[1]	test-error:0.404738
[2]	test-error:0.36437
[3]	test-error:0.34097
[4]	test-error:0.331918
[5]	test-error:0.32475
[6]	test-error:0.319754
[7]	test-error:0.315512
[8]	test-error:0.311284
[9]	test-error:0.308898
[10]	test-error:0.307206
[11]	test-error:0.304674
[12]	test-error:0.303676
[13]	test-error:0.30237
[14]	test-error:0.30134
[15]	test-error:0.299422
[16]	test-error:0.298594
[17]	test-error:0.297768
[18]	test-error:0.29728
[19]	test-error:0.296026
[20]	test-error:0.294522
[21]	test-error:0.293628
[22]	test-error:0.293202
[23]	test-error:0.291962
[24]	test-error:0.291472
[25]	test-error:0.291148
[26]	test-error:0.290512
[27]	test-error:0.289894
[28]	test-error:0.289394
[29]	test-error:0.288766
[30]	test-error:0.288244
[31]	test-error:0.288022
[32]	test-error:0.287828
[33]	test-error:0.287544
[34]	test-error:0.287298
[35]	test-error:0.28721
[36]	test-error:0.286594
[37]	test-error:0.2861
[38]	test-error:0.285792
[39]	test-error:0.2851